In [2]:
import pandas as pd
import os
import requests
from lxml import html
from collections import defaultdict
import time

In [3]:
df = pd.read_csv('collection_catalog.csv', sep=';')

In [4]:
def tempf(x):
    try:
        res = int(x)
        return(x)
    except:
        return -1

In [6]:
nik = pd.read_csv('selected_books_a10nik.csv', sep=',', usecols=['cat_id', 'mukhin_year', 'polka_title'])
nik.dropna(subset=['cat_id'], inplace=True)
nik['cat_id'] = nik['cat_id'].apply(tempf).astype(int)
nik = nik[nik['cat_id'] > 0]
df = pd.read_csv('liter.csv', sep=';')
catalog = pd.read_csv('collection_catalog.csv', sep=';')
liter = set(df['произведение '])
cat = set(catalog['book_title'])

In [7]:
nik.at[200, 'mukhin_year'] = 1927
nik.drop([307], axis=0, inplace=True)
nik[nik['cat_id'] == 105610]


,cat_id,mukhin_year,polka_title
200,105610,1927,Двенадцать стульев


In [8]:
tempdf = catalog.set_index('fbusta_id').join(nik.set_index('cat_id'), how='left').rename_axis('fbusta_id').reset_index()

In [9]:
tempdf.shape

(645, 9)

In [15]:
withdates = tempdf
#.reset_index().set_index('book_title').join(df[['произведение ', 'год написания']].set_index('произведение '), how='left').reset_index()

In [16]:
withdates

,fbusta_id,author_id,author_name,author_surname,book_id,book_title,size,mukhin_year,polka_title
0,158030,0,Фёдор Александрович,Абрамов,1,Братья и сестры,247,1958,NaN
1,158033,0,Фёдор Александрович,Абрамов,2,Две зимы и три лета,293,1968,NaN
2,158036,0,Фёдор Александрович,Абрамов,3,Пути-перепутья,254,1973,NaN
3,158032,0,Фёдор Александрович,Абрамов,4,Дом,242,1978,NaN
4,144729,1,Борис,Акунин,5,Нефритовые четки,575,2006,NaN
5,305713,1,Борис,Акунин,6,Турецкий гамбит,172,1998,NaN
6,461230,1,Борис,Акунин,7,Статский советник,250,1999,NaN
7,131819,1,Борис,Акунин,8,Алтын-Толобас,314,2001,NaN
8,191882,1,Борис,Акунин,9,Ф. М.,422,2006,NaN
9,70938,2,Светлана Александровна,Алексиевич,10,У войны - не женское лицо…,256,1983,NaN


In [73]:
# withdates.rename(columns={
#     'level_0': 'book_title',
#     'index': 'fbusta_id',
# }, inplace=True)

In [74]:
# withdates['mukhin_year'] = withdates['mukhin_year'].fillna(withdates['год написания'])

In [17]:
with open('Книги • Полка.html', 'r') as f:
    content = f.read()
tree = html.fromstring(content)
section = tree.xpath('//*[@id="app-root"]/div/main/div[5]')
s = 0
rows = []
while True:
    try:
        title = section[0][s][0][1][0].text_content()
        author = section[0][s][0][1][1][0].text_content()
        year = section[0][s][0][1][1][1].text_content()
    except:
        break
    s += 1
    rows.append((year, title, author))
# print(year)
# print(section[0][2][0][1][1][1].text_content())


In [18]:
polka_years = pd.DataFrame(rows[:-1])

In [19]:
polka_years = polka_years[polka_years[0] != 'Илья Ильф']
polka_years = polka_years[polka_years[0] != 'Евгений Петров']


In [20]:
withdates = withdates.set_index('polka_title').join(polka_years.set_index(1), how='left').reset_index()

In [21]:
withdates.shape

(645, 11)

In [22]:
withdates

,index,fbusta_id,author_id,author_name,author_surname,book_id,book_title,size,mukhin_year,0,2
0,Архипелаг ГУЛАГ,308939,124,Александр Исаевич,Солженицын,558,Архипелаг ГУЛаг,1724,1958-1968,1968,Александр Солженицын
1,Бедная Лиза,143864,60,Николай Михайлович,Карамзин,265,Бедная Лиза,15,NaN,1792,Николай Карамзин
2,Белая гвардия,261423,20,Михаил Афанасьевич,Булгаков,83,Белая гвардия,249,1923 - 1924,1925,Михаил Булгаков
3,Бесы,169991,32,Федор Михайлович,Достоевский,139,Бесы,648,1872,1872,Фёдор Достоевский
4,Борис Годунов,173154,106,Александр Сергеевич,Пушкин,477,Борис Годунов,112,NaN,1825,Александр Пушкин
5,Братья Карамазовы,492702,32,Федор Михайлович,Достоевский,140,Братья Карамазовы,947,1880,1880,Фёдор Достоевский
6,Былое и думы,168751,48,Александр Иванович,Герцен,202,Былое и думы,652,NaN,1852,Александр Герцен
7,В овраге,169265,26,Антон Павлович,Чехов,120,В овраге,36,1899,1900,Антон Чехов
8,Вечера на хуторе близ Диканьки,74286,50,Николай Васильевич,Гоголь,220,Вечера на хуторе близ Диканьки,197,NaN,1832,Николай Гоголь
9,Вишнёвый сад,124471,26,Антон Павлович,Чехов,121,Вишневый сад,50,NaN,1903,Антон Чехов


In [23]:
withdates['mukhin_year'] = withdates['mukhin_year'].fillna(withdates[0])

In [24]:
import numpy as np
import re

In [25]:
def year_transform(x):
    if x is not np.nan:
        numbers = re.findall('\d+',str(x))
#         print(x, numbers)
        return max(map(int, numbers))
    return x

In [26]:
withdates['mukhin_year'] = withdates['mukhin_year'].apply(year_transform).astype('Int64')

In [27]:
withdates.rename(columns={
    'index': 'polka_title',
    'mukhin_year': 'year'
}, inplace=True)

In [28]:
withdates

,polka_title,fbusta_id,author_id,author_name,author_surname,book_id,book_title,size,year,0,2
0,Архипелаг ГУЛАГ,308939,124,Александр Исаевич,Солженицын,558,Архипелаг ГУЛаг,1724,1968,1968,Александр Солженицын
1,Бедная Лиза,143864,60,Николай Михайлович,Карамзин,265,Бедная Лиза,15,1792,1792,Николай Карамзин
2,Белая гвардия,261423,20,Михаил Афанасьевич,Булгаков,83,Белая гвардия,249,1924,1925,Михаил Булгаков
3,Бесы,169991,32,Федор Михайлович,Достоевский,139,Бесы,648,1872,1872,Фёдор Достоевский
4,Борис Годунов,173154,106,Александр Сергеевич,Пушкин,477,Борис Годунов,112,1825,1825,Александр Пушкин
5,Братья Карамазовы,492702,32,Федор Михайлович,Достоевский,140,Братья Карамазовы,947,1880,1880,Фёдор Достоевский
6,Былое и думы,168751,48,Александр Иванович,Герцен,202,Былое и думы,652,1852,1852,Александр Герцен
7,В овраге,169265,26,Антон Павлович,Чехов,120,В овраге,36,1899,1900,Антон Чехов
8,Вечера на хуторе близ Диканьки,74286,50,Николай Васильевич,Гоголь,220,Вечера на хуторе близ Диканьки,197,1832,1832,Николай Гоголь
9,Вишнёвый сад,124471,26,Антон Павлович,Чехов,121,Вишневый сад,50,1903,1903,Антон Чехов


In [29]:
withdates = withdates[[
    'author_id', 'author_name', 'author_surname',
    'book_id', 'book_title', 'fbusta_id', 'polka_title',
    'size', 'year']].sort_values(['author_id', 'book_id']).reset_index(drop=True)

In [30]:
withdates['year'] = withdates['year'].astype('Int64')

In [31]:
withdates

,author_id,author_name,author_surname,book_id,book_title,fbusta_id,polka_title,size,year
0,0,Фёдор Александрович,Абрамов,1,Братья и сестры,158030,NaN,247,1958
1,0,Фёдор Александрович,Абрамов,2,Две зимы и три лета,158033,NaN,293,1968
2,0,Фёдор Александрович,Абрамов,3,Пути-перепутья,158036,NaN,254,1973
3,0,Фёдор Александрович,Абрамов,4,Дом,158032,NaN,242,1978
4,1,Борис,Акунин,5,Нефритовые четки,144729,NaN,575,2006
5,1,Борис,Акунин,6,Турецкий гамбит,305713,NaN,172,1998
6,1,Борис,Акунин,7,Статский советник,461230,NaN,250,1999
7,1,Борис,Акунин,8,Алтын-Толобас,131819,NaN,314,2001
8,1,Борис,Акунин,9,Ф. М.,191882,NaN,422,2006
9,2,Светлана Александровна,Алексиевич,10,У войны - не женское лицо…,70938,NaN,256,1983


In [45]:
withdates.to_csv('collection_catalog_full.csv', sep=';', index=False)

In [42]:
surname_counts = withdates[['author_id', 'author_surname']].drop_duplicates()['author_surname'].value_counts()
surnames = surname_counts[surname_counts > 1].index

In [48]:
surnames

Index(['Алешковский', 'Быков', 'Успенский', 'Андреев', 'Толстой', 'Ерофеев'], dtype='object')

In [124]:
withdates[(withdates['mukhin_year'].isna()) & ~(withdates['polka_title'].isna())]

,index,author_id,author_name,author_surname,book_id,size,mukhin_year,polka_title,год написания
25,Бедная Лиза,60,Николай Михайлович,Карамзин,265,15,NaN,Бедная Лиза,NaN
43,Борис Годунов,106,Александр Сергеевич,Пушкин,477,112,NaN,Борис Годунов,NaN
54,Былое и думы,48,Александр Иванович,Герцен,202,652,NaN,Былое и думы,NaN
72,Вечера на хуторе близ Диканьки,50,Николай Васильевич,Гоголь,220,197,NaN,Вечера на хуторе близ Диканьки,NaN
77,Вишневый сад,26,Антон Павлович,Чехов,121,50,NaN,Вишнёвый сад,NaN
79,Во сне ты горько плакал,65,Юрий Павлович,Казаков,293,17,NaN,Во сне ты горько плакал,NaN
112,Голубая книга,145,Михаил Михайлович,Зощенко,644,289,NaN,Голубая книга,NaN
133,Дама с собачкой,26,Антон Павлович,Чехов,115,16,NaN,Дама с собачкой,NaN
177,Евгений Онегин,106,Александр Сергеевич,Пушкин,480,182,NaN,Евгений Онегин,NaN
202,Записки из подполья,32,Федор Михайлович,Достоевский,143,117,NaN,Записки из подполья,NaN


In [58]:
nik = pd.read_csv('selected_books_a10nik.csv', sep=',', usecols=['cat_title', 'cat_id', 'mukhin_wiki1', 'mukhin_wiki2', 'cat_f', 'cat_i' ,'cat_id'])
nik.dropna(subset=['cat_id'], inplace=True)
nik['cat_id'] = nik['cat_id'].apply(tempf).astype(int)
nik = nik[nik['cat_id'] > 0]
nik.at[200, 'mukhin_year'] = 1927
nik.drop([307], axis=0, inplace=True)
nik[nik['cat_id'] == 105610]

full = pd.read_csv('collection_catalog_full.csv', sep=';') \
.set_index('fbusta_id').join(nik[['cat_id', 'mukhin_wiki2']].set_index("cat_id")).reset_index() \
.rename(columns={
    'index': 'fbusta_id',
    'mukhin_wiki2': 'wiki_book',
})

In [47]:
full

,fbusta_id,author_id,author_name,author_surname,book_id,book_title,polka_title,size,year,wiki_book
0,158030,0,Фёдор Александрович,Абрамов,1,Братья и сестры,NaN,247,1958.0,https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%...
1,158033,0,Фёдор Александрович,Абрамов,2,Две зимы и три лета,NaN,293,1968.0,NaN
2,158036,0,Фёдор Александрович,Абрамов,3,Пути-перепутья,NaN,254,1973.0,NaN
3,158032,0,Фёдор Александрович,Абрамов,4,Дом,NaN,242,1978.0,NaN
4,144729,1,Борис,Акунин,5,Нефритовые четки,NaN,575,2006.0,https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D1%...
5,305713,1,Борис,Акунин,6,Турецкий гамбит,NaN,172,1998.0,https://ru.wikipedia.org/wiki/%D0%A2%D1%83%D1%...
6,461230,1,Борис,Акунин,7,Статский советник,NaN,250,1999.0,https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%...
7,131819,1,Борис,Акунин,8,Алтын-Толобас,NaN,314,2001.0,https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D1%...
8,191882,1,Борис,Акунин,9,Ф. М.,NaN,422,2006.0,https://ru.wikipedia.org/wiki/%D0%A4._%D0%9C.
9,70938,2,Светлана Александровна,Алексиевич,10,У войны - не женское лицо…,NaN,256,1983.0,https://ru.wikipedia.org/wiki/%D0%A3_%D0%B2%D0...


In [59]:
a = nik[(nik['cat_f'] == 'Быков') & (nik['cat_i'] == 'Василь')].index
b = nik[(nik['cat_f'] == 'Толстой') & (nik['cat_i'] == 'Лев')].index
c = nik[(nik['cat_f'] == 'Успенский') & (nik['cat_i'] == 'Глеб')].index
d = nik[(nik['cat_f'] == 'Алешковский') & (nik['cat_i'] == 'Юз')].index
e = nik[(nik['cat_f'] == 'Ерофеев') & (nik['cat_i'] == 'Виктор')].index
f = nik[(nik['cat_f'] == 'Андреев') & (nik['cat_i'] == 'Леонид')].index
g = nik[(nik['cat_f'] == 'Некрасов') & (nik['cat_i'] == 'Николай')].index

drop_idx = [*a, *b, *c, *d, *e, *f, *g]
nik.drop(drop_idx, axis=0, inplace=True)
# nik.dropna(subset=['mukhin_wiki1'])['cat_f'].value_counts()

In [60]:
full = full.set_index('author_surname').join(
    nik[['cat_f', 'mukhin_wiki1']].set_index('cat_f').dropna(subset=['mukhin_wiki1'])) \
.reset_index().rename(columns={
    'index': 'author_surname',
    'mukhin_wiki1': 'wiki_author',
})

# print(full.shape)
# full = full[[
#     'author_id', 'author_name', 'author_surname',
#     'book_id', 'book_title', 'fbusta_id', 'polka_title',
#     'size', 'year', 'wiki_author', 'wiki_book']].sort_values(['author_id', 'book_id']).reset_index(drop=True)
# full.shape

In [61]:
full

,author_surname,fbusta_id,author_id,author_name,book_id,book_title,polka_title,size,year,wiki_book,wiki_author
0,Абрамов,158030,0,Фёдор Александрович,1,Братья и сестры,NaN,247,1958.0,https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%...,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
1,Абрамов,158033,0,Фёдор Александрович,2,Две зимы и три лета,NaN,293,1968.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
2,Абрамов,158036,0,Фёдор Александрович,3,Пути-перепутья,NaN,254,1973.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
3,Абрамов,158032,0,Фёдор Александрович,4,Дом,NaN,242,1978.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
4,Аверченко,159215,9,Аркадий Тимофеевич,40,Дюжина ножей в спину революции,NaN,39,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
5,Аверченко,169499,9,Аркадий Тимофеевич,41,О маленьких – для больших,NaN,123,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
6,Аверченко,159231,9,Аркадий Тимофеевич,42,Подходцев и двое других,NaN,128,1917.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
7,Аверченко,417321,9,Аркадий Тимофеевич,43,Разумная экономия,NaN,5,1912.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
8,Аверченко,159241,9,Аркадий Тимофеевич,44,Шутка мецената,NaN,105,1925.0,https://ru.wikipedia.org/wiki/%D0%A8%D1%83%D1%...,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
9,Айтматов,150859,10,Чингиз,45,Белый пароход,NaN,113,1970.0,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%...,https://ru.wikipedia.org/wiki/%D0%90%D0%B9%D1%...


In [71]:
full[(full['author_surname'] == 'Андреев')].groupby('wiki_author').size()

wiki_author
https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B5%D0%B2,_%D0%9B%D0%B5%D0%BE%D0%BD%D0%B8%D0%B4_%D0%9D%D0%B8%D0%BA%D0%BE%D0%BB%D0%B0%D0%B5%D0%B2%D0%B8%D1%87    5
dtype: int64

In [64]:
a = full[(full['author_surname'] == 'Быков') & (full['author_name'] == 'Василь')].index
b = full[(full['author_surname'] == 'Толстой') & (full['author_name'] == 'Лев Николаевич')].index
c = full[(full['author_surname'] == 'Успенский') & (full['author_name'] == 'Глеб Иванович')].index
d = full[(full['author_surname'] == 'Алешковский') & (full['author_name'] == 'Юз')].index
e = full[(full['author_surname'] == 'Ерофеев') & (full['author_name'] == 'Виктор Владимирович')].index
f = full[(full['author_surname'] == 'Андреев') & (full['author_name'] == 'Леонид Николаевич')].index

# nik.drop([*a, *b, *c], axis=0, inplace=True)
# full[(full['author_surname'] == 'Толстой')]
for i in a:
    full.at[i, 'wiki_author'] = 'https://ru.wikipedia.org/wiki/%D0%91%D1%8B%D0%BA%D0%BE%D0%B2,_%D0%92%D0%B0%D1%81%D0%B8%D0%BB%D1%8C_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87'
for i in b:
    full.at[i, 'wiki_author'] = 'https://ru.wikipedia.org/wiki/%D0%A2%D0%BE%D0%BB%D1%81%D1%82%D0%BE%D0%B9,_%D0%9B%D0%B5%D0%B2_%D0%9D%D0%B8%D0%BA%D0%BE%D0%BB%D0%B0%D0%B5%D0%B2%D0%B8%D1%87'
for i in c:
    full.at[i, 'wiki_author'] = 'https://ru.wikipedia.org/wiki/%D0%A3%D1%81%D0%BF%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B9,_%D0%93%D0%BB%D0%B5%D0%B1_%D0%98%D0%B2%D0%B0%D0%BD%D0%BE%D0%B2%D0%B8%D1%87'
for i in d:
    full.at[i, 'wiki_author'] = 'https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B5%D1%88%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9,_%D0%AE%D0%B7'
for i in e:
    full.at[i, 'wiki_author'] = 'https://ru.wikipedia.org/wiki/%D0%95%D1%80%D0%BE%D1%84%D0%B5%D0%B5%D0%B2,_%D0%92%D0%B8%D0%BA%D1%82%D0%BE%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87'
for i in f:
    full.at[i, 'wiki_author'] = 'https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B5%D0%B2,_%D0%9B%D0%B5%D0%BE%D0%BD%D0%B8%D0%B4_%D0%9D%D0%B8%D0%BA%D0%BE%D0%BB%D0%B0%D0%B5%D0%B2%D0%B8%D1%87'

In [75]:
full.to_csv('collection_catalog_full.csv', sep=';', index=False)

In [110]:
row = []
for l in liter:
    if l not in cat:
        mx = 0
        cur = None
        for c in cat:
            d = similar(l, c)
            if d > mx:
                cur = c
                mx = d
        row.append((round(mx, 3), l, cur))

In [112]:
sorted(row)

[(0.336,
  'Птицы, или Новые сведения о человеке (Птицы, или Оглашение человека)',
  'Двойник или мои вечера в Малороссии (Часть 1)'),
 (0.382,
  'Лампа Мафусаила, или Крайняя битва чекистов с масонами',
  'Черная курица, или Подземные жители'),
 (0.384,
  '1993. Семейный портрет на фоне горящего дома \n',
  'Мой брат играет на кларнете'),
 (0.39,
  'Жизнь и необычайные приключения солдата Ивана Чонкина',
  'Ботинки, полные горячей водки'),
 (0.393, 'Шекспир мне друг, но истина дороже', 'Незнайка в Солнечном городе'),
 (0.4, 'Общага-на-Крови', 'Оправдание'),
 (0.407, 'Они сражались за Родину (главы)', 'В дни поражений и побед'),
 (0.41, 'Не соответствует критериям', 'Четвертый Рим'),
 (0.423, 'Город Глупов в последние десять лет', 'Голубое и зеленое'),
 (0.435, 'Любка Казак', 'Пиковая Дама'),
 (0.435, 'Шерри-Бренди', 'Шум времени'),
 (0.444, 'Как это делалось в Одессе', 'Как закалялась сталь'),
 (0.449, 'Понедельник начинается в субботу', 'Пикник на обочине'),
 (0.449, 'Рваные валенки 

In [19]:
print(len(cat))
print(len(liter))
print(len(cat.intersection(liter)))

644
541
353


In [95]:
for r in (([r[2] for r in row if r[0] > 0.840])):
    if r not in last:
        print(r)

Номер Один, или В садах других возможнос­тей


In [ ]:
 (0.565, 'Generation „П“» («Поколение „П“)', 'Generation «П»'),
 (0.667, 'А…..а', 'А.....а'),
 (0.7, 'Чернобыльская молитва', 'Чернобыльская молитва. Хроника будущего'),
 (0.732,
  'Приключения майора Звягина',
  'Приключения майора Звягина.  Роман воспитания'),
(0.75, 'Батальоны просят огня', 'Батальоны просят огня (редакция №1)'),
 (0.75,
  'Синенький скромный платочек',
  'Синенький скромный платочек. Скорбная повесть'),
    

In [20]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [21]:
liter

{' Институт сновидений',
 ' Крепость',
 '1993. Семейный портрет на фоне горящего дома \n',
 '2017',
 '38 попугаев',
 'Generation „П“» («Поколение „П“)',
 'А зори здесь тихие…',
 'Авиатор',
 'Акимунды',
 'Алтын-Толобас',
 'Алферьев',
 'Алые паруса',
 'Ангелочек',
 'Андеграунд, или Герой нашего времени',
 'Анна Каренина',
 'Антоновские яблоки',
 'Аристократка',
 'Архипелаг ГУЛАГ',
 'Асан',
 'Асфальт',
 'Аэлита',
 'А…..а',
 'Баня',
 'Басурман',
 'Батальоны просят огня',
 'Бегущая по волнам',
 'Безумная Евдокия',
 'Белая гвардия',
 'Белая голубка Кордовы',
 'Белеет парус одинокий',
 'Белоснежка и принц',
 'Белые одежды',
 'Белый Бим Черное ухо',
 'Белый пароход',
 'Берег',
 'Бермудский треугольник черной вдовы',
 'Бессмертный',
 'Бесы',
 'Благая весть',
 'Блистающий мир',
 'Бог Х',
 'Богиня прайм-тайма',
 'Богомолье',
 'Братья Карамазовы',
 'Братья и сестры',
 'Бронзовая птица',
 'Будьте готовы, Ваше высочество',
 'В дни поражений и побед',
 'В дурном обществе',
 'В круге первом',
 'В овра

In [24]:
similar('Generation „П“» («Поколение „П“)', 'Generation «П»')

0.5652173913043478